In [16]:
from boto.s3.connection import S3Connection
from ipyparallel import Client
import json
from collections import Counter

In [2]:
# Start your engines!
rc = Client()
engines = rc[:]
len(engines)

10

In [26]:
## ensure that every engine has the same functions available
with engines.sync_imports():
    from boto.s3.connection import S3Connection
    import json


importing S3Connection from boto.s3.connection on engine(s)
importing json on engine(s)


In [27]:
## connect to aws
s3 = S3Connection()
bucket = s3.get_bucket('twitter-streaming02062017')
list_of_aws_keys = [str(i.name) for i in bucket.list()]

In [33]:

%%px 
def twitter_scatter(individual_key):
    """Create a client to s3. Then, iterate through the keys in the given bucket. Return a counter of each hashtag."""
    s3 = S3Connection()
    bucket = s3.get_bucket('twitter-streaming02062017',validate=False)
    key_file = bucket.get_key(individual_key)
    strFile = key_file.get_contents_as_string()
    result = []
    count = 0
    for tweet in strFile.split('/n'):
        try:
            entity = json.loads(tweet)
            try:
                entity = entity['entities']
                if entity != None and (entity['hashtags'] != None or len(entity['hashtags']) != 0):
                    for hashtags in entity['hashtags']:
                        result.append(hashtags['text'])
            except KeyError :
                count+=1
        except ValueError:
            count+=1
    return result



In [34]:
print(len(list_of_aws_keys)) ## Number of files to go through
## scatter the keys to each engine
engines.scatter('s3' , list_of_aws_keys).get()
%px y = [twitter_scatter(_) for _ in s3]


27


In [35]:
## next, ge the results of parsing the json from each engine
result = engines.gather('y')
from functools import reduce
top_results = Counter(list(result)[0])



In [36]:
#print out the top 100
top_results.most_common(n=100)

[(u'CD9', 393),
 (u'OneDBestFans', 116),
 (u'CamilaBestFans', 108),
 (u'iHeartAwards', 51),
 (u'BestFanArmy', 45),
 (u'KCAPinoyStar', 42),
 (u'\uc0ac\uc124\ud1a0\ud1a0\uc0ac\uc774\ud2b8\ucd94\ucc9c', 41),
 (u'\u0e25\u0e38\u0e07\u0e27\u0e34\u0e28\u0e27\u0e30', 33),
 (u'LizaSoberano', 31),
 (u'KCAEstrellaLatina', 27),
 (u'\u0632\u064a\u0632\u0648_HBD', 27),
 (u'BestFans2017', 26),
 (u'quote', 23),
 (u'NavoiyDay', 22),
 (u'KCAPersonalidadeBR', 20),
 (u'6DaysUntilMyExAndWhys', 20),
 (u'ChainedToTheRhythm', 19),
 (u'BIGOLIVE', 19),
 (u'ShePersisted', 19),
 (u'Lovatics', 18),
 (u'\u0628\u0636\u0627\u064a\u0642_\u0644\u0645\u0627', 18),
 (u'\u062a\u064a\u0645_\u0627\u0644\u063a\u0627\u0628\u0647', 18),
 (u'ALDUB82ndWeeksary', 17),
 (u'\u0628\u0641\u0643\u0631_\u0627\u0644\u0635\u0631\u062e\u064a_\u0633\u064a\u0646\u062a\u0635\u0631_\u0627\ufef7\u0633\u0644\u0627\u0645',
  16),
 (u'\u0627\u0644\u0641\u0631\u0633\u0627\u0646_\u0648\u062d\u0641\u0644\u0629_\u0627\u0644\u0641\u0648\u0644\u0648\u0